In [1]:
from loaddata import DataTrainLoader

In [2]:
import torch
import numpy as np
from tqdm import tqdm

In [3]:
from torchvision import transforms

In [4]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(size=(224, 224)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(), 
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])
                                      ])

In [5]:
full_dr = DataTrainLoader(path="data", transform=train_transforms)

In [6]:
train_size = int(0.8 * len(full_dr))
test_size = len(full_dr) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dr, [train_size, test_size])

In [7]:
train = torch.utils.data.DataLoader(train_dataset, batch_size=1000, shuffle=True)

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5
    plt.imshow(np.transpose(img, (1, 2, 0)))

In [9]:
label_tags = ['0', '1']

In [10]:
dataiter = iter(train)
images, labels = dataiter.next()
images = images.numpy()

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(label_tags[labels[idx].int()])

In [11]:
test = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=True)
dataiter = iter(test)
images, labels = dataiter.next()
images = images.numpy() 

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(label_tags[labels[idx].int()])

In [12]:
import torchvision.models as models

In [13]:
model = models.resnet50(pretrained=True)

In [14]:
from torch import nn
from torch import optim

In [16]:
for param in model.parameters():
    param.requires_grad = False
from collections import OrderedDict
fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2048, 1000)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(1000, 1)),
    ('output', nn.Sigmoid())
]))
model.fc = fc

In [17]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [18]:
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.BCELoss().to('cuda')

In [20]:

# number of epochs to train the model
n_epochs = 30

test_loss_min = np.Inf # track change in testation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and testation loss
    train_loss = 0.0
    test_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in tqdm(train):
        data, target = data.cuda(), target.cuda().float()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # testate the model #
    ######################
    model.eval()
    for data, target in tqdm(test):
        data, target = data.cuda(), target.cuda().float()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average testation loss 
        test_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train.dataset)
    test_loss = test_loss/len(test.dataset)
        
    # print training/testation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \t test Loss: {:.6f}'.format(
        epoch, train_loss, test_loss))
    
    # save model if testation loss has decreased
    if test_loss <= test_loss_min:
        print('testation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        test_loss_min,
        test_loss))
        torch.save(model.state_dict(), 'model_cifar'+str(epoch)+'.pt')
        test_loss_min = test_loss

In [21]:
test_result_transforms = transforms.Compose([transforms.Resize(size=(224, 224)),
                                             transforms.ToTensor(), 
                                             transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                            ])

In [22]:
from loaddata import DataTestLoader

In [23]:
test_result = DataTestLoader(path='data', transform=test_result_transforms)

In [24]:
test = torch.utils.data.DataLoader(test_result, batch_size=1000)

In [42]:
model.load_state_dict(torch.load('model_cifar27.pt'))
import pandas as pd

In [ ]:
model.eval()
main_preds = pd.DataFrame()
for data, target in tqdm(test):
    data = data.cuda()
    output = model(data)
    output = output.to('cpu').detach().numpy()
    output = np.where(output > 0.5, 1, 0)
    output = output[:,-1]
    test_preds = pd.DataFrame({'id': target, 'label': output})
    main_preds+= test_preds

 86%|████████▌ | 50/58 [06:11<00:59,  7.44s/it]